## Libraries

In [2]:
%pip install surprise
from surprise.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
from surprise import Dataset
from surprise import Reader
import pandas as pd
import numpy as np

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=2811600 sha256=9be8a36069782d7b37e8ea6814a799aaa009014c837f2bc698db85120c7c8f27
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [13]:
ratings = pd.read_csv('similar users.csv')

In [14]:
ratings.head(10)

,ITEM,ProductId,Rating,UserId
0,Anarkali kurti,375884478,5,AO83WU65LG
1,Party wear boots,342947458,3,A4K9I17GSZ
2,Denim Shirt,967465275,2,A4K9I17GSZ
3,High Rise jeans,997762297,3,A4K9I17GSZ
4,High Rise jeans,997762297,3,A2M6KY5LEQ
5,High Rise jeans,997762297,1,A2M6KY5LEQ
6,Round neck sweater,472746972,5,A2M6KY5LEQ
7,Synthetic Sandals,538354264,5,A2M6KY5LEQ
8,Running Shoes,251319821,1,A2M6KY5LEQ
9,Denim Shirt,967465275,3,AN4ZADBUL3


### No of Uers and Products

In [15]:
# Total
n_users = ratings.UserId.shape[0]
n_items = ratings.ProductId.shape[0]

print(n_users,n_items)

54 54


In [16]:
# Unique
n_users = ratings.UserId.unique().shape[0]
n_items = ratings.ProductId.unique().shape[0]

print(n_users,n_items)

11 9


### Users who have rated more than 2 times

In [19]:
x = ratings['UserId'].value_counts()>4
x

ACEWOS1AUB     True
AYLPYXDAQA     True
AGQWD7V3GI     True
AV6QHTO59N     True
A2M6KY5LEQ     True
A7E28IX8V4     True
AN4ZADBUL3    False
A5FP6GE8YM    False
A4K9I17GSZ    False
ASM7XBP7PA    False
AO83WU65LG    False
Name: UserId, dtype: bool

In [20]:
y = x[x].index
y

Index(['ACEWOS1AUB', 'AYLPYXDAQA', 'AGQWD7V3GI', 'AV6QHTO59N', 'A2M6KY5LEQ',
       'A7E28IX8V4'],
      dtype='object')

In [21]:
ratings = ratings[ratings['UserId'].isin(y)]

In [22]:
ratings.head

<bound method NDFrame.head of                   ITEM  ProductId  Rating      UserId
4      High Rise jeans  997762297       3  A2M6KY5LEQ
5      High Rise jeans  997762297       1  A2M6KY5LEQ
6   Round neck sweater  472746972       5  A2M6KY5LEQ
7    Synthetic Sandals  538354264       5  A2M6KY5LEQ
8        Running Shoes  251319821       1  A2M6KY5LEQ
13   Rugged Cargo pant  221566227       3  AYLPYXDAQA
14  Round neck sweater  472746972       4  AYLPYXDAQA
15    Party wear boots  342947458       1  AYLPYXDAQA
16    Party wear boots  342947458       5  AYLPYXDAQA
17       Running Shoes  251319821       2  AYLPYXDAQA
18    Oversized Tshirt  475224575       3  AYLPYXDAQA
19     High Rise jeans  997762297       2  AYLPYXDAQA
20         Denim Shirt  967465275       3  AGQWD7V3GI
21  Round neck sweater  472746972       4  AGQWD7V3GI
22       Running Shoes  251319821       1  AGQWD7V3GI
23      Anarkali kurti  375884478       3  AGQWD7V3GI
24    Oversized Tshirt  475224575       1  AGQWD7V3G

In [23]:
no_of_ratings = ratings.groupby('ITEM')['Rating'].count().reset_index()

In [24]:
no_of_ratings.rename(columns={'Rating':'No. of Ratings'},inplace = True)
no_of_ratings

,ITEM,No. of Ratings
0,Anarkali kurti,3
1,Denim Shirt,3
2,High Rise jeans,6
3,Oversized Tshirt,3
4,Party wear boots,3
5,Round neck sweater,7
6,Rugged Cargo pant,6
7,Running Shoes,4
8,Synthetic Sandals,5


In [25]:
final_ratings = ratings.merge(no_of_ratings, on='ITEM')
final_ratings

,ITEM,ProductId,Rating,UserId,No. of Ratings
0,High Rise jeans,997762297,3,A2M6KY5LEQ,6
1,High Rise jeans,997762297,1,A2M6KY5LEQ,6
2,High Rise jeans,997762297,2,AYLPYXDAQA,6
3,High Rise jeans,997762297,5,AV6QHTO59N,6
4,High Rise jeans,997762297,2,ACEWOS1AUB,6
5,High Rise jeans,997762297,4,ACEWOS1AUB,6
6,Round neck sweater,472746972,5,A2M6KY5LEQ,7
7,Round neck sweater,472746972,4,AYLPYXDAQA,7
8,Round neck sweater,472746972,4,AGQWD7V3GI,7
9,Round neck sweater,472746972,2,AGQWD7V3GI,7


In [ ]:
# IGNORE THIS CELL
# final_ratings = final_ratings[final_ratings['No. of Ratings']>=90]
# final_ratings.shape

In [26]:
final_ratings.drop_duplicates(['ITEM','UserId'],inplace=True)
final_ratings.shape

(30, 5)

In [27]:
item_pivot = final_ratings.pivot_table(columns = 'UserId', index = 'ITEM', values = 'Rating')
item_pivot.fillna(0, inplace = True)
item_pivot

UserId,A2M6KY5LEQ,A7E28IX8V4,ACEWOS1AUB,AGQWD7V3GI,AV6QHTO59N,AYLPYXDAQA
ITEM,,,,,,
Anarkali kurti,0.0,0.0,3.0,3.0,0.0,0.0
Denim Shirt,0.0,5.0,2.0,3.0,0.0,0.0
High Rise jeans,3.0,0.0,2.0,0.0,5.0,2.0
Oversized Tshirt,0.0,0.0,0.0,1.0,1.0,3.0
Party wear boots,0.0,0.0,1.0,0.0,0.0,1.0
Round neck sweater,5.0,1.0,0.0,4.0,0.0,4.0
Rugged Cargo pant,0.0,4.0,4.0,0.0,3.0,3.0
Running Shoes,1.0,0.0,0.0,1.0,1.0,2.0
Synthetic Sandals,5.0,1.0,5.0,0.0,1.0,0.0


In [28]:
item_sparse = csr_matrix(item_pivot)
item_sparse

<9x6 sparse matrix of type '<class 'numpy.float64'>'
	with 30 stored elements in Compressed Sparse Row format>

### Model

In [29]:
model = NearestNeighbors(algorithm='brute')

In [30]:
model.fit(item_sparse)

NearestNeighbors(algorithm='brute')

In [32]:
distances, suggestions = model.kneighbors(item_pivot.loc['Round neck sweater', : ].values.reshape(1,-1), n_neighbors = 3)

In [33]:
print(type(item_pivot.loc['Denim Shirt', : ]))

<class 'pandas.core.series.Series'>


In [34]:
suggestions

array([[5, 7, 3]])

In [35]:
#suggestions for turkey by similar users
for i in range(len(suggestions)):
    print(item_pivot.index[suggestions[i]])

Index(['Round neck sweater', 'Running Shoes', 'Oversized Tshirt'], dtype='object', name='ITEM')


### Saving the model and the pivot table

In [36]:
#Saving Pivot table
import joblib
pkl_file = open("item_pivot.pkl","wb")
joblib.dump(item_pivot,pkl_file)
pkl_file.close()

In [37]:
#Saving Model
import joblib
pkl_file = open("similarUser.pkl","wb")
joblib.dump(model,pkl_file)
pkl_file.close()

In [38]:
#Load Model and pivot table
data = joblib.load('similarUser.pkl')
pt = joblib.load('item_pivot.pkl')

In [39]:
#Try Outs
a = 'Denim Shirt'
distances, suggestions = data.kneighbors(pt.loc[a, : ].values.reshape(1,-1), n_neighbors = 3)

In [40]:
for i in range(len(suggestions)):
    print(pt.index[suggestions[i]])

Index(['Denim Shirt', 'Anarkali kurti', 'Rugged Cargo pant'], dtype='object', name='ITEM')
